In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('../src/train.csv')
X = df.iloc[:,:-1].drop('ID',axis=1)
y = df.iloc[:,-1]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
X_train.shape,y_train.shape

((60816, 369), (60816,))

In [2]:
cat_col = X_train.select_dtypes(include = ['object','category']) # 0 для этого датасета
Q1 = X_train.quantile(0.25)
Q3 = X_train.quantile(0.75)
IQR = Q3 - Q1
X1 = X_train[~((X_train < (Q1 - 1.5 * IQR)) | (X_train > (Q3 + 1.5 * IQR)))]
X_train = X1.fillna(X1.mean())

In [3]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler # сделали очистку от выбросов, теперь делаем проверку на Varience Threshold
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)
thr = VarianceThreshold(threshold = 0.05)
thr.fit(X_scaled,y)
cols = X_scaled.columns[thr.get_support()] # get_support() делает то, что даёт нужные столбцы
X_thr = pd.DataFrame(thr.transform(X_scaled),columns = cols)
X_thr.shape # ну, здесь всё +- нормально уже.

(60816, 25)

In [5]:
X_corr = X_thr.corr() # это - такой же DataFrame
hold = 0.8
to_drop = []
for col in X_corr.columns:
    cor_features = X_corr[col].abs()>hold # у Series вместо индексов - строки, но они ничего не делают. Берет абсолютные значения и смотрит у каждого, есть ли у него корреляция больше нужной
    cor_features[col] = False # не учитывается корреляция с самим собой
    if any(cor_features):  # Если есть хоть один признак, коррелирующий с текущим выше порога.
        to_drop.append(col)
X_final = X_thr.drop(columns = to_drop)
X_final.shape,len(to_drop) # оказалось вообще смешно

((60816, 17), 8)

In [6]:
# X_thr - после Threshold, X_final - уже когда после корреляции
cols = X_final.columns
X_test = pd.DataFrame(scaler.fit_transform(X_test[X_scaled.columns]),columns=X_scaled.columns)[cols]

In [7]:
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(min_samples_leaf = 4,random_state=42)
model.fit(X_final,y_train)
X_final = X_final[X_final.columns[model.feature_importances_>0.05]]
X_test = X_test[X_final.columns]

In [8]:
final_model = RandomForestClassifier(min_samples_leaf = 4) # здесь уже можно делать валидацию разных моделей
final_model.fit(X_final,y_train)
score = final_model.score(X_test, y_test)
print(f'{score} при 5 из 369 признаков'.center(80,'🎯'))

🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯0.9596816627203367 при 5 из 369 признаков🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
